In [33]:
mode = "Live" # Backtesting/Live/Algo

%run "KiteConnect_Library.ipynb"   
%run "KiteConnect_Strategies.ipynb" 

warnings.filterwarnings('ignore')



In [2]:
#%run "KiteConnect_Strategies.ipynb" 
STATE = ""
openpositions = 0
tradelogger.setLevel(logging.DEBUG)

%%javascript
IPython.notebook.kernel.execute("URL = '" + window.location + "'");

In [3]:
portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['BHARTIARTL'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

In [4]:
portfolio[0].iloc[-1]

'BHARTIARTL'

In [5]:
papertrade = True

In [5]:
if mode == "Live":
    kws.connect(threaded=True)

In [6]:
if mode == "Live":
    print(kws.is_connected())

True


In [8]:
if mode == "Live":
    kws.close()

In [110]:
symbol = 'BHARTIARTL'
symbolToken = EQTOKEN('BHARTIARTL')

price = kite.ltp(symbolToken)[str(symbolToken)]['last_price']
toTick = lambda x,n=5: np.round((np.floor(x *100)+n-1)/n)*n/100

sl = toTick(price * 0.001)
target = toTick(price * 0.01)

In [111]:
papertrade = False
buy_bo(symbol, price=price, trigger=price, stoploss=sl, squareoff=target, quantity=1)

In [ ]:
papertrade = False
sell_bo(symbol, price, price, sl, target, 1)

In [92]:
price-sl

315.05

In [50]:
orders_df = pd.DataFrame(kite.orders())
open_orders = orders_df[(orders_df['tradingsymbol']=='BHARTIARTL') & (orders_df['status'] != 'REJECTED')  & (orders_df['tag'] == 'bot')]



In [108]:
parent_order_id = open_orders['parent_order_id'].iloc[0]
order_id = open_orders['order_id'].iloc[0]

In [109]:
kite.exit_order(variety=kite.VARIETY_BO, order_id=order_id, parent_order_id=parent_order_id)



In [51]:
open_orders

,average_price,cancelled_quantity,disclosed_quantity,exchange,exchange_order_id,exchange_timestamp,exchange_update_timestamp,filled_quantity,guid,instrument_token,...,product,quantity,status,status_message,tag,tradingsymbol,transaction_type,trigger_price,validity,variety
4,0,0,0,NSE,None,None,None,0,tLWRkvVO69mG1BKq,2714625,...,BO,1,REJECTED,Order was placed outside of trading hours (ada...,bot,BHARTIARTL,BUY,320,DAY,bo


In [32]:
orders_df = pd.DataFrame(kite.orders())
open_orders = orders_df[(orders_df['tradingsymbol']=='BHARTIARTL') & (orders_df['status'] != 'REJECTED')  & (orders_df['tag'] == 'bot')]
for index, row in open_orders.iterrows():
    print(row.order_id, row.parent_order_id)
    kite.exit_order(variety=kite.VARIETY_AMO, order_id=row.order_id, parent_order_id=row.parent_order_id)

190429000023899 None


In [36]:
papertrade = False
buy_bo('BHARTIARTL',price=320, trigger=320, stoploss=1,squareoff=10,tag="bot")